In [2]:
# Format Data for CNN and NN

import numpy as np
import os
import pickle
from utils import *
import torch

train_dir = 'FeaturesTrain'
# test_dir = 'FeatruesTest'


DATA = []
d = train_dir
for i in os.listdir(d):
    with open(os.path.join(d,i), 'rb') as f:
        DATA.append(pickle.load(f))


from sklearn.model_selection import train_test_split


x_keys = ['mfccCoeffs', 'chromaCoeffs','contrastCoeffs', 'tonnetz','melspectCoeffs']

Y = np.c_[[emotion_labels[i['emotion']] for i in DATA]].flatten()
X = np.c_[[np.concatenate([standard_scale[j](i[j]) for j in x_keys]).reshape(1,-1) for i in DATA]]
x_train, x_val, y_train, y_val = train_test_split(X, Y, train_size=0.8, stratify=Y)


from imblearn.over_sampling import RandomOverSampler 

idxs = np.arange(len(x_train)).reshape(-1,1)
ros = RandomOverSampler(random_state=42)
rs_idx, y_idx = ros.fit_resample(idxs, y_train)
rs_idx = rs_idx.flatten()
x_train = x_train[rs_idx]
y_train = y_train[rs_idx]


from torch.utils.data import Dataset, DataLoader
AD_train = AudioData(x_train, y_train)
AD_val = AudioData(x_val, y_val)

trainloader = DataLoader(AD_train, batch_size=8, shuffle=True)
valloader = DataLoader(AD_val, batch_size=8, shuffle=True)


device = torch.device('cpu')

## NN

In [3]:
from models import NN
import torch

nn = NN(141860)
nn.load_state_dict(torch.load('models/nn.pt'))
nn.eval()


NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=141860, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Linear(in_features=512, out_features=512, bias=True)
    (6): ReLU()
    (7): Linear(in_features=512, out_features=8, bias=True)
  )
)

In [4]:
from utils import confusionMatrix

confusionMatrix(nn, valloader)

array([[31,  0,  0,  0,  0,  1,  0,  0],
       [ 0, 30,  0,  0,  1,  0,  1,  0],
       [ 1,  0, 28,  1,  0,  0,  1,  1],
       [ 1,  0,  0, 29,  1,  0,  0,  1],
       [ 0,  0,  0,  2, 30,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 15,  0,  0],
       [ 0,  1,  0,  0,  2,  0, 28,  1],
       [ 0,  0,  0,  2,  0,  0,  0, 30]])

## CNN

In [5]:
from models import ConvNet
import torch


cnn = ConvNet(173)
cnn.load_state_dict(torch.load('models/cnn.pt'))
cnn.eval()

ConvNet(
  (convnet): Sequential(
    (0): Conv1d(1, 128, kernel_size=(5,), stride=(1,), padding=same)
    (1): ReLU()
    (2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=same)
    (4): ReLU()
    (5): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=same)
    (7): ReLU()
    (8): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
    (10): Linear(in_features=283648, out_features=512, bias=True)
    (11): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): Linear(in_features=512, out_features=8, bias=True)
  )
)

In [6]:
from utils import confusionMatrix

confusionMatrix(cnn, valloader)

array([[31,  0,  0,  0,  0,  1,  0,  0],
       [ 0, 32,  0,  0,  0,  0,  0,  0],
       [ 1,  0, 29,  1,  0,  0,  1,  0],
       [ 0,  0,  0, 32,  0,  0,  0,  0],
       [ 2,  0,  0,  0, 30,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 15,  0,  0],
       [ 0,  0,  0,  2,  0,  2, 28,  0],
       [ 0,  0,  0,  1,  0,  0,  0, 31]])